In [2]:
import fitz
import io
from PIL import Image
import dotenv 
import google.generativeai as genai

c:\Users\99013030\Desktop\katha_diagram\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
file = "./files/first_chapter.pdf"

page_start = 1
page_end = 3
# file = "./files/2024-dbir-data-breach-investigations-report.pdf"
# file = "./files/two_chapters.pdf"
pdf_file = fitz.open(file)

for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    image_list = page.get_images()

    if image_list:
        print(f"[+] Found a total of {len(image_list)} images in page {page_index}")
    
    for image_index, img in enumerate(image_list, start=1):
        xref = img[0]
        base_image = pdf_file.extract_image(xref)
        image_bytes = base_image["image"]
        image_ext = base_image["ext"]
        
        image = Image.open(io.BytesIO(image_bytes))
        image.save(open(f"./files/images/image{page_index + 1}_{image_index}.{image_ext}", "wb"))

[+] Found a total of 2 images in page 2
[+] Found a total of 1 images in page 3
[+] Found a total of 1 images in page 4


c:\Users\99013030\Desktop\katha_diagram\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from pdf2image import convert_from_path

pil_images = convert_from_path('files/first_chapter.pdf')

In [3]:

api_key = dotenv.get_key('.env', key_to_get='GEMINI_API_KEY')

#Configure API key
genai.configure(api_key=api_key)

In [5]:
gen_config = genai.GenerationConfig(
    temperature=0.5,
)


model = genai.GenerativeModel(
    model_name='gemini-1.5-flash-latest',
    generation_config=gen_config
)

extracted_text = ""

for img in pil_images:
    extracted_text += model.generate_content([
        f"""Given a page in the pdf, summarize the content topic wise. Return the answer in the following method
            pdf = FPDF(orientation='P', unit='mm', format='A4')
            pdf.add_page()
            pdf.set_font('helvetica', 'bold', 10)
            pdf.set_text_color(255, 255, 255)
            pdf.cell(0, 10, 'Hello, World!', 1, 1, 'C')
            pdf.image('path_to_your_image.jpg', x=10, y=10, w=50, h=50)
            pdf.output('output.pdf', 'F')""", img]
    ).text

print(extracted_text)

KeyboardInterrupt: 